# main

In [1]:
import json
import requests
import popelines
import os
from main import fix_values, populating_vals, flatten, flatten_dupe_vals
from datetime import datetime

In [2]:
def fix_values(value, key, reset_key):
    if key == reset_key:
        new_list = []
        for x in value:
            value[x][f'{reset_key}_id'] = x
            new_list.append(value[x])
        return new_list
    else:
        return value

In [3]:
# read endpoint, returns a json file of the HTTP request
def read_endpoint(endpoint, headers_set, params_set=None):
    try:
        response = requests.get(endpoint, headers=headers_set, params=params_set)
        response_text = json.loads(response.text)
        response.raise_for_status()

    except requests.exceptions.HTTPError as err:
        print(err)

    return response_text

In [4]:
# # generate all experiments given a project id
# def generate_experiments(project_id, experiment_endpoint, experiment_headers, experiment_params):
#     # get all experiments from one project
#     j_exp = read_endpoint(endpoint=experiment_endpoint, headers_set=experiment_headers, params_set=experiment_params)

#     # loop for all experiments in this project
# #     experiment_id_list = []
# #     upload_exp_list = []
# #     for exp in j_exp:
# #         exp['project_id'] = project_id
# #         upload_exp_list.append(flatten(exp, {}, ''))
# #         experiment_id_list.append(exp['id'])
        
#     return upload_exp_list, experiment_id_list

In [5]:
# generate all projects within account
def generate_projects(project_endpoint, project_headers):
    # get all projects
    j_proj = read_endpoint(endpoint=project_endpoint, headers_set=project_headers)

    # store a list of project metadata
    return j_proj

# load all projects, get project_id's

In [6]:
gbq_key = os.environ.get('GOOGLE_ACCOUNT_CREDENTIALS')
pope = popelines.popeline(dataset_id='optimizely', service_key_file_loc=gbq_key, directory='.', verbose=False)

# Optimizely parameters
headers = {
    'Authorization': 'Bearer 2:EWAWmaXb4TgtYVU2VvwoEF-9UbJxBahkiFh1633_Oc9nmju7iJis',
    'Content-Type': 'application/json',
}

In [ ]:
project_endpoint = 'https://api.optimizely.com/v2/projects'

In [ ]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
# pope.write_to_json(file_name='../uploads/projects.json', jayson=all_projects, mode='w')

In [ ]:
# pope.write_to_bq(table_name='projects', file_name='../uploads/projects.json', append=True, ignore_unknown_values=False, bq_schema_autodetect=False)
# print("Successfully uploaded all projects.")

In [ ]:
project_id_list = []
for project in all_projects:
    project_id_list.append((project['id']))

In [ ]:
# project_id_list


In [ ]:
# project_id_name = []
# for project in all_projects:
#     project_id_name.append((project['id'], project['name']))

In [ ]:
# project_id_name

# load experiments

In [7]:
experiment_endpoint = 'https://api.optimizely.com/v2/experiments'


In [45]:
# experiment_id_list = []
# all_exp_list = []

# for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
params = (
    ('project_id', 9965963792),
    ('per_page', 100),
) 


In [46]:
exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)



In [10]:
experiment_id_list = []

In [11]:
exp_id_list = []
for exp in exp_list:
    exp_id_list.append(exp['id'])

In [12]:
experiment_id_list.extend(exp_id_list)

In [47]:
new = []

In [73]:
for e in exp_list:
    if e['id'] == 11039523400:
        new = e

In [74]:
exp=new

In [75]:
exp

{'allocation_policy': 'manual',
 'audience_conditions': 'everyone',
 'campaign_id': 11041413439,
 'changes': [],
 'created': '2018-07-07T22:18:03.362400Z',
 'description': 'Switching "Email" field to say "Company Email" - will lead quality increase and will form conversion be impacted?',
 'earliest': '2018-07-07T22:25:27.209360Z',
 'holdback': 0,
 'id': 11039523400,
 'is_classic': False,
 'last_modified': '2018-07-10T15:45:19.606040Z',
 'latest': '2018-07-08T18:29:55.140960Z',
 'metrics': [{'aggregator': 'unique',
   'event_id': 10757331770,
   'scope': 'visitor',
   'winning_direction': 'increasing'},
  {'aggregator': 'unique',
   'event_id': 10601774395,
   'scope': 'visitor',
   'winning_direction': 'increasing'},
  {'aggregator': 'sum',
   'field': 'revenue',
   'scope': 'visitor',
   'winning_direction': 'increasing'}],
 'name': 'FT Sign Up Company Email Field July 7',
 'page_ids': [11094440334],
 'project_id': 9965963792,
 'status': 'paused',
 'type': 'a/b',
 'variations': [{'act

In [76]:
all_singles = []
nested_key_list = []
for k,v in exp.items():
    if isinstance(v, list):
        nested_key_list.append(k)

single_layer_experiment = {}    
for k,v in exp.items():
    if k not in nested_key_list:
        k = k.replace('-', '_')
        single_layer_experiment[k] = exp[k]
single_layer_experiment['upload_ts'] = str(datetime.now())


all_singles.append(flatten(single_layer_experiment, {}, ''))


In [77]:
all_singles

[{'allocation_policy': 'manual',
  'audience_conditions': 'everyone',
  'campaign_id': 11041413439,
  'created': '2018-07-07T22:18:03.362400Z',
  'description': 'Switching "Email" field to say "Company Email" - will lead quality increase and will form conversion be impacted?',
  'earliest': '2018-07-07T22:25:27.209360Z',
  'holdback': 0,
  'id': 11039523400,
  'is_classic': False,
  'last_modified': '2018-07-10T15:45:19.606040Z',
  'latest': '2018-07-08T18:29:55.140960Z',
  'name': 'FT Sign Up Company Email Field July 7',
  'project_id': 9965963792,
  'status': 'paused',
  'type': 'a/b',
  'upload_ts': '2019-02-20 15:05:27.285798'}]

In [81]:
# nested part into separate tables:
# metrics table:
 
flattened_metric = []
for element in exp['metrics']:
    flattened_metric.append(element)

updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

metric_list = []
# for metric in new_flattened_metric:
#     print(metric)
#     metric_dict = {}
#     metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
#     metric_dict['metrics_event_id'] = metric['metrics_event_id']
#     metric_dict['metrics_scope'] =  metric['metrics_scope']
#     metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
#     metric_dict['experiment_id'] = exp['id']
#     metric_dict['upload_ts'] = str(datetime.now())
#     metric_list.append(metric_dict)


{'allocation_policy': 'manual', 'audience_conditions': 'everyone', 'campaign_id': 11041413439, 'created': '2018-07-07T22:18:03.362400Z', 'description': 'Switching "Email" field to say "Company Email" - will lead quality increase and will form conversion be impacted?', 'earliest': '2018-07-07T22:25:27.209360Z', 'holdback': 0, 'id': 11039523400, 'is_classic': False, 'last_modified': '2018-07-10T15:45:19.606040Z', 'latest': '2018-07-08T18:29:55.140960Z', 'metrics_aggregator': 'unique', 'metrics_event_id': 10757331770, 'metrics_scope': 'visitor', 'metrics_winning_direction': 'increasing', 'name': 'FT Sign Up Company Email Field July 7', 'project_id': 9965963792, 'status': 'paused', 'type': 'a/b', 'variations_archived': False, 'variations_name': 'Label says "Business Email"', 'variations_status': 'active', 'variations_variation_id': 11047562271, 'variations_weight': 3334}
{'allocation_policy': 'manual', 'audience_conditions': 'everyone', 'campaign_id': 11041413439, 'created': '2018-07-07T22

In [84]:
new_flattened_metric

[{'allocation_policy': 'manual',
  'audience_conditions': 'everyone',
  'campaign_id': 11041413439,
  'created': '2018-07-07T22:18:03.362400Z',
  'description': 'Switching "Email" field to say "Company Email" - will lead quality increase and will form conversion be impacted?',
  'earliest': '2018-07-07T22:25:27.209360Z',
  'holdback': 0,
  'id': 11039523400,
  'is_classic': False,
  'last_modified': '2018-07-10T15:45:19.606040Z',
  'latest': '2018-07-08T18:29:55.140960Z',
  'metrics_aggregator': 'unique',
  'metrics_event_id': 10757331770,
  'metrics_scope': 'visitor',
  'metrics_winning_direction': 'increasing',
  'name': 'FT Sign Up Company Email Field July 7',
  'project_id': 9965963792,
  'status': 'paused',
  'type': 'a/b',
  'variations_archived': False,
  'variations_name': 'Label says "Business Email"',
  'variations_status': 'active',
  'variations_variation_id': 11047562271,
  'variations_weight': 3334},
 {'allocation_policy': 'manual',
  'audience_conditions': 'everyone',
  

In [87]:
metrics_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    # single layer fields:
    nested_key_list = []
    for k,v in exp.items():
        if isinstance(v, list) or isinstance(v, dict):
            nested_key_list.append(k)

    single_layer_experiment = {}    
    for k,v in exp.items():
        if k not in nested_key_list:
            k = k.replace('-', '_')
            single_layer_experiment[k] = exp[k]
    single_layer_experiment['upload_ts'] = str(datetime.now())

    all_singles.append(flatten(single_layer_experiment, {}, ''))

    # nested part into separate tables:
    # metrics table:

    flattened_metric = []
    for element in exp['metrics']:
        flattened_metric.append(element)

    updated_metric = populating_vals(outer_dict=exp, inner_flattened_list=flattened_metric, destination_key='metrics')
    new_flattened_metric = flatten_dupe_vals(vals=updated_metric, key='metrics')

    metric_list = []
    for metric in new_flattened_metric:
        metric_dict = {}
        metric_dict['metrics_aggregator'] = metric['metrics_aggregator']
        if 'metrics_event_id' in metric.keys():
            metric_dict['metrics_event_id'] = metric['metrics_event_id']
        metric_dict['metrics_scope'] =  metric['metrics_scope']
        metric_dict['metrics_winning_direction'] = metric['metrics_winning_direction']
        metric_dict['experiment_id'] = exp['id']
        metric_dict['upload_ts'] = str(datetime.now())
        metric_list.append(metric_dict)
        
    metrics_table.extend(metric_list)
    

Processing experiment 10756790433
Processing experiment 10796678057
Processing experiment 10800885728
Processing experiment 10815382176
Processing experiment 10945903729
Processing experiment 10955300529
Processing experiment 11039523400
Processing experiment 11938601024
Processing experiment 11966791385
Processing experiment 11992110033


In [205]:
variations_table = []
for exp in exp_list:
    print(f"Processing experiment {exp['id']}")

    variations = {}
    # for exp in exp_list:

    print(f"Processing experiment {exp['id']}")

    variations['experiment_id'] = exp['id']
    variations['variations'] = exp['variations']

    flattened_variations = []

    for var in exp['variations']:
        flattened_actions = []
        if len(var['actions']) > 0:
            for action in var['actions']:
                flattened_changes = []
                for element in action['changes']:
                    flattened_changes.append(element)
                # Replace old 'changes' with new 'flattened_changes'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

                update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
                flat = flatten_dupe_vals(vals=update_actions, key='actions')
                flattened_actions.extend(flat)

        else:
            other_flat = {}
            for k,v in var.items():
                if k != 'actions':
                    other_flat['actions'] = []
                    other_flat[k] = v
            flat = [other_flat]
            flattened_actions.extend(flat)

        update_variations = populating_vals(outer_dict=variations, inner_flattened_list=flattened_actions, destination_key='variations')
        flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

    variations_table.extend(flattened_variations)
    
    

Processing experiment 10756790433
Processing experiment 10756790433
Processing experiment 10796678057
Processing experiment 10796678057
Processing experiment 10800885728
Processing experiment 10800885728
Processing experiment 10815382176
Processing experiment 10815382176
Processing experiment 10945903729
Processing experiment 10945903729
Processing experiment 10955300529
Processing experiment 10955300529
Processing experiment 11039523400
Processing experiment 11039523400
Processing experiment 11938601024
Processing experiment 11938601024
Processing experiment 11966791385
Processing experiment 11966791385
Processing experiment 11992110033
Processing experiment 11992110033


In [209]:
for v in variations_table:
    if v['experiment_id'] == 10945903729:
        print(v['variations_name'])


Carousel
No Carousel


In [143]:
exp['variations']

[{'actions': [],
  'archived': False,
  'name': 'Original',
  'status': 'active',
  'variation_id': 11994110041,
  'weight': 5000},
 {'actions': [{'changes': [{'async': False,
      'attributes': {'href': 'https://freetrial.infusionsoft.com/signup/2'},
      'css': {},
      'dependencies': [],
      'id': 'CAE0BD33-7160-429C-BA16-A6A7E73F558A',
      'rearrange': {'insertSelector': '', 'operator': 'before'},
      'selector': '.homepage-hero-ft-cta',
      'type': 'attribute'}],
    'page_id': 11933141690,
    'share_link': 'http://www.infusionsoft.com?optimizely_token=1023b14998b0975f176cb8cf234dca953e6cb5016d957a058d1a8c7652d7d3fc&optimizely_x=11970581627&optimizely_x_audiences=&optimizely_preview_layer_ids=11935062681&optimizely_snippet=s3-9965963792'}],
  'archived': False,
  'name': 'Redirect FT CTA to freetrial.infusionsoft.com/signup/2',
  'status': 'active',
  'variation_id': 11970581627,
  'weight': 5000}]

In [63]:
exp['url_targeting']

{'activation_type': 'url_changed',
 'conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/setup"}, {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/signup"}]]',
 'edit_url': 'freetrial.infusionsoft.com/setup',
 'key': '9965963792_url_targeting_for_ft_signup_variations',
 'page_id': 10752650977}

In [58]:
expanded_exp

{'experiment_id': 10756790433,
 'url_targeting_activation_type': 'url_changed',
 'url_targeting_conditions': '["and", ["or", {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/setup"}, {"match_type": "simple", "type": "url", "value": "freetrial.infusionsoft.com/signup"}]]',
 'url_targeting_edit_url': 'freetrial.infusionsoft.com/setup',
 'url_targeting_key': '9965963792_url_targeting_for_ft_signup_variations',
 'url_targeting_page_id': 10752650977}

In [ ]:
# expanded_exp['experiment_id'] = exp['id']
# expanded_exp['variations'] = exp['variations']


In [ ]:
flattened_variations = []

for var in expanded_exp['variations']:
    flattened_actions = []
    for action in var['actions']:
        print(action)
#         flattened_changes = []
#         for element in action['changes']:
#             flattened_changes.append(element)

        # Replace old 'timeseries' with new 'flattened_timeseries'
#         updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
#         new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

#     update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
#     flattened_actions.extend(flatten_dupe_vals(vals=update_actions, key='actions'))  

# update_variations = populating_vals(outer_dict=expanded_exp, inner_flattened_list=flattened_actions, destination_key='variations')
# flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

# expanded_variations = flattened_variations

In [ ]:
flattened_changes

## One table for all single layer fields

In [ ]:
nested_key_list = []

In [ ]:
for k,v in exp_list[0].items():
    if isinstance(v, list) or isinstance(v, dict):
        nested_key_list.append(k)

In [ ]:
nested_key_list

In [ ]:
single_layer_experiment = {}

In [ ]:
for k,v in exp_list[0].items():
    if k not in nested_key_list:
        single_layer_experiment[k] = exp_list[0][k]
        

In [ ]:
single_layer_experiment

In [ ]:
all_singles = []

In [ ]:
all_singles.append(single_layer_experiment)

In [ ]:
all_singles

## Expand nested part manually into separate tables:

In [ ]:
expanded_j_list = []

In [ ]:
for exp in j_exp:

    expanded_exp = {}
    
    expanded_exp['experiment_id'] = exp['id']
    expanded_exp['changes'] = exp['changes']

    expanded_metrics = {}
    expanded_metrics['experiment_id'] = exp['id']
    expanded_metrics['metrics'] = exp['metrics']
    expanded_exp['metrics'] = flatten_dupe_vals(vals=expanded_metrics, key='metrics')

    if 'url_targeting' in exp.keys():
        expanded_exp['experiment_id'] = exp['id']
        expanded_exp['url_targeting'] = exp['url_targeting']
        expanded_exp = flatten(expanded_exp, {}, '')
    
    expanded_exp['experiment_id'] = exp['id']
    expanded_exp['variations'] = exp['variations']
    
    flattened_variations = []
    
    for var in expanded_exp['variations']:
        flattened_actions = []
        for action in var['actions']:
            flattened_changes = []
            for element in action['changes']:
                flattened_changes.append(element)

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
            new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

        update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
        flattened_actions.extend(flatten_dupe_vals(vals=update_actions, key='actions'))  

    update_variations = populating_vals(outer_dict=expanded_exp, inner_flattened_list=flattened_actions, destination_key='variations')
    flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))
    
    expanded_variations = flattened_variations
    
    expanded_exp = expanded_variations
    expanded_j_list.extend(expanded_exp)
    

In [ ]:
type(expanded_j_list[12]['variations_actions_changes_value'])

# results timeseries

In [ ]:
# experiment_id_list

In [ ]:
# example of no results
# result_endpoint = 'https://api.optimizely.com/v2/experiments/11479333433/timeseries'
# example of normal time series
result_endpoint = 'https://api.optimizely.com/v2/experiments/11477653122/timeseries'
# example of 
# result_endpoint = f'https://api.optimizely.com/v2/experiments/{experiment_id}/timeseries'



In [ ]:
response_ts = requests.get(result_endpoint, headers=headers)

# if '' then the experiment has not started yet
if response_ts.text == '':
    j_ts = {'experiment_id': experiment_id}
elif 'bad' in response_ts.text:
    j_ts = {'experiment_id': experiment_id}
else:
    j_ts = json.loads(response_ts.text)

new_j_ts = pope.fix_json_values(callback=fix_values, obj=j_ts, reset_key='results')


In [ ]:
flattened_j_ts = []

flattened_metrics = []
for metric in new_j_ts['metrics']:
    if 'results' in metric.keys():
        for ts in metric['results']:
            flattened_timeseries = []
            for element in ts['timeseries']:
                element['upload_ts'] = str(datetime.now())
                flattened_timeseries.append(flatten(element, {}, ''))

            # Replace old 'timeseries' with new 'flattened_timeseries'
            updated_results = populating_vals(outer_dict=ts, inner_flattened_list=flattened_timeseries, destination_key='timeseries')
            flattened_results = flatten_dupe_vals(vals=updated_results, key='timeseries')

        # Replace old 'metrics' with new 'flattened_results'
        update_metrics = populating_vals(outer_dict=metric, inner_flattened_list=flattened_results, destination_key='results')
        flattened_metrics.extend(flatten_dupe_vals(vals=update_metrics, key='results'))

    else:
        flattened_metrics = [flatten(new_j_ts, {}, '')]

update_new_j_ts = populating_vals(outer_dict=new_j_ts, inner_flattened_list=flattened_metrics, destination_key='metrics')
flattened_j_ts.extend(flatten_dupe_vals(vals=update_new_j_ts, key='metrics'))



In [ ]:
j_ts

In [ ]:
flattened_j_ts

In [ ]:
new_j_ts['metrics']


            

# testing

In [ ]:
all_projects = generate_projects(project_endpoint, headers)

In [ ]:
len(all_projects)

In [ ]:
project_id_list = []
for project in all_projects:
    project_id_list.append(project['id'])

# to accumulate all experiment_id, for 
experiment_id_list = []

expanded_exp_list = []
all_singles = []
# loop over all project_id_list to get experiments within each project
for project_id in project_id_list:
    # params include project_id (required) and experiments pulling per each request (default only 25)
    params = (
        ('project_id', project_id),
        ('per_page', 100),
    ) 

    exp_list = read_endpoint(endpoint=experiment_endpoint, headers_set=headers, params_set=params)
    exp_id_list = []
    for exp in exp_list:
        exp_id_list.append(exp['id'])
    experiment_id_list.extend(exp_id_list)

    ###### Game plan is to separate nested fields from single layer fields, upload them to separate table, then do joins on BQ level
#     expanded_exp_list = []
#     all_singles = []

    for exp in exp_list:
        print(f"Processing experiment {exp['id']}, of project {project_id}")

        # single layer fields:
        nested_key_list = []
        for k,v in exp.items():
            if isinstance(v, list) or isinstance(v, dict):
                nested_key_list.append(k)

        single_layer_experiment = {}    
        for k,v in exp.items():
            if k not in nested_key_list:
                k = k.replace('-', '_')
                single_layer_experiment[k] = exp[k]

        all_singles.append(single_layer_experiment)

        # nested part into separate tables:
        expanded_exp = {}

        expanded_exp['experiment_id'] = exp['id']
        expanded_exp['changes'] = exp['changes']

        expanded_metrics = {}
        expanded_metrics['experiment_id'] = exp['id']
        expanded_metrics['metrics'] = exp['metrics']
        expanded_exp['metrics'] = flatten_dupe_vals(vals=expanded_metrics, key='metrics')

        if 'url_targeting' in exp.keys():
            expanded_exp['experiment_id'] = exp['id']
            expanded_exp['url_targeting'] = exp['url_targeting']
            expanded_exp = flatten(expanded_exp, {}, '')

        expanded_exp['experiment_id'] = exp['id']
        expanded_exp['variations'] = exp['variations']

        flattened_variations = []

        for var in expanded_exp['variations']:
            flattened_actions = []
            for action in var['actions']:
                flattened_changes = []
                for element in action['changes']:
                    flattened_changes.append(element)

                # Replace old 'timeseries' with new 'flattened_timeseries'
                updated_changes = populating_vals(outer_dict=action, inner_flattened_list=flattened_changes, destination_key='changes')
                new_flattened_changes = flatten_dupe_vals(vals=updated_changes, key='changes')

            update_actions = populating_vals(outer_dict=var, inner_flattened_list=new_flattened_changes, destination_key='actions')
            flattened_actions.extend(flatten_dupe_vals(vals=update_actions, key='actions'))  

        update_variations = populating_vals(outer_dict=expanded_exp, inner_flattened_list=flattened_actions, destination_key='variations')
        flattened_variations.extend(flatten_dupe_vals(vals=update_variations, key='variations'))

        expanded_variations = flattened_variations

        expanded_exp = expanded_variations
        expanded_exp_list.extend(expanded_exp)
        
        

In [ ]:
for e in expanded_exp_list:
    print(e.keys())